# Testing

In [1]:
import docker
client = docker.from_env()

In [6]:
client.images.list()

[<Image: 'autonomicbsc/workflow_api:latest', 'my-model:latest'>,
 <Image: 'autonomicbsc/workflow_master:latest'>,
 <Image: 'autonomicbsc/workflow_server:latest'>,
 <Image: 'autonomicbsc/workflow_worker:latest'>,
 <Image: 'autonomicbsc/cluster_hadoop_spark_python:latest'>,
 <Image: 'gusseppe/sparkhadoopcluster:latest'>,
 <Image: 'ubuntu:16.04'>,
 <Image: 'mlflow-docker-example-c11e2a0:latest'>,
 <Image: 'mlflow-docker-example:latest'>,
 <Image: 'continuumio/miniconda3:latest'>,
 <Image: 'drp:latest'>,
 <Image: 'ubuntu:latest'>,
 <Image: 'gusseppe/catnip:latest'>,
 <Image: 'jupyter/pyspark-notebook:latest'>,
 <Image: 'continuumio/miniconda:4.5.4'>,
 <Image: 'python:3-onbuild'>,
 <Image: 'prakhar1989/static-site:latest'>]

In [7]:
client.images.pull('hello-world')

[<Image: 'hello-world:latest', 'hello-world:linux'>]

In [15]:
container = client.containers.run('hello-world', detach=True)
container.id

'4f7ee4f89dc59100bdc45d400407b54a7f1f4d6b540f3e78adebe01fd10a2e10'

In [23]:
container.status

'created'

In [25]:
client.containers.get(container.id).logs()

b'\nHello from Docker!\nThis message shows that your installation appears to be working correctly.\n\nTo generate this message, Docker took the following steps:\n 1. The Docker client contacted the Docker daemon.\n 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.\n    (amd64)\n 3. The Docker daemon created a new container from that image which runs the\n    executable that produces the output you are currently reading.\n 4. The Docker daemon streamed that output to the Docker client, which sent it\n    to your terminal.\n\nTo try something more ambitious, you can run an Ubuntu container with:\n $ docker run -it ubuntu bash\n\nShare images, automate workflows, and more with a free Docker ID:\n https://hub.docker.com/\n\nFor more examples and ideas, visit:\n https://docs.docker.com/get-started/\n\n'

In [26]:
container = client.containers.run('ubuntu', name='ubuntu_ctn', 
                                  command="/bin/bash", tty=True,detach=True)

# Rewritting

In [30]:
!ls

api_docker.py  Dockerfile.app_base    docker_python_sdk.ipynb  scripts
build.sh       Dockerfile.app_master  LICENSE		       start.sh
config	       Dockerfile.app_server  requirements.txt	       steps_deploy_api
data	       Dockerfile.app_worker  run_api		       stop.sh


In [11]:
import os
base_path = '/home/guess/Desktop/autodeploy/examples/workflow_sklearn'
path = os.path.join(base_path, 'Dockerfile')

In [13]:
image = client.images.build(path=base_path,tag='worker_test')

In [17]:

image = client.images.build(path=base_path, dockerfile=path, tag='worker_test')

In [22]:
os.path.dirname(path)

'/home/guess/Desktop/autodeploy/examples/workflow_sklearn'

In [14]:
image[0]

<Image: 'hello-world:latest', 'worker_test:latest'>

In [54]:
# client.images.remove('worker_test')
client.images.prune()

{'ImagesDeleted': None, 'SpaceReclaimed': 0}

In [56]:
client.containers.prune()

{'ContainersDeleted': ['0f19c70aa3f842b4b51448b71a75d8a60d5823ab5b57c4fe244842b70b764ae4',
  'f4efd3446d9f34a88e5bdbdf2540de49653946095d3c1ad382d5c209328fd86f',
  '4f7ee4f89dc59100bdc45d400407b54a7f1f4d6b540f3e78adebe01fd10a2e10',
  'b8a8cac4693dc09148bd0867718c1f465d5ac30018fc04a73053c74c07cdcac1',
  'bb8762cf0f1234fad343fa5c5f544f59b6e669bef3103b8bccee361f11757318',
  '152f8ee650b6edce423cc994bd35d6705defdf3160a0528a07df7eccfcd1dafb',
  '986610c57ad2234eab304fe7a747629bd542aef5d4816b90502dc28997974650',
  '6a87ac9cf9790564006949ddc6cbcf96e34b19062385f0b2d97bc6b83e6efb91',
  '626d8db20490d874f753e069a18ffe1e2314f29d8e67d835b4155b7574b20b8d',
  '1f560b17389da9f0c30e3a156950154a54db8d227943b8477959c19a842b6865'],
 'SpaceReclaimed': 2075907165}

In [61]:
client.images.list()

[<Image: 'autonomicbsc/workflow_api:latest', 'my-model:latest'>,
 <Image: 'autonomicbsc/workflow_master:latest'>,
 <Image: 'autonomicbsc/workflow_server:latest'>,
 <Image: 'autonomicbsc/workflow_worker:latest'>,
 <Image: 'autonomicbsc/cluster_hadoop_spark_python:latest'>,
 <Image: 'gusseppe/sparkhadoopcluster:latest'>,
 <Image: 'ubuntu:16.04'>,
 <Image: 'mlflow-docker-example-c11e2a0:latest'>,
 <Image: 'mlflow-docker-example:latest'>,
 <Image: 'continuumio/miniconda3:latest'>,
 <Image: 'drp:latest'>,
 <Image: 'ubuntu:latest'>,
 <Image: 'gusseppe/catnip:latest'>,
 <Image: 'jupyter/pyspark-notebook:latest'>,
 <Image: 'hello-world:latest', 'hello-world:linux'>,
 <Image: 'continuumio/miniconda:4.5.4'>,
 <Image: 'python:3-onbuild'>,
 <Image: 'prakhar1989/static-site:latest'>]

In [66]:
client.containers.list()[0].stop()

In [68]:
client.images.remove('fce289e99eb9', force=True)

In [139]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

# Author: Gusseppe Bravo <gbravor@uni.pe>
# License: BSD 3 clause

import docker
import errno

client = docker.from_env()

class Setup:
    def __init__(self,
                 single_file,
                 master_file=None,
                 worker_file=None,
                 n_workers=2,
                 type_app='single'):
        """
        Example:
            setup = Setup(single_file='/home/user/Dockerfile')

        Parameters:
            single_file (str): Path to a Dockerfile, single mode.
            master_file (str): Path to a Dockerfile, cluster mode.
            worker_file (str): Path to a Dockerfile, cluster mode.
            n_workers (int): if type_app=='cluster' then it means the 
                             number of slaves (workers)
            type_app (str): Type of platform.
        
        """    
        self.type_app = type_app
        self.single_file = single_file
        self.master_file = master_file
        self.worker_file = worker_file
        self.n_workers = n_workers
        self.tag_image = None
        self.image = None
        self.containers = None
        
    def build(self, name='app'):
        """
        Build a platform with Docker images.
        
        Parameters:
            name (str): Prefix of a Docker image.
        Returns:
            image (object): Docker image.
        """
        if self.type_app == 'single':
            base_path = os.path.dirname(self.single_file)
            self.tag_image = name+'_'+self.type_app
            image = client.images.build(path=base_path, 
                                        dockerfile=self.single_file, 
                                        tag=self.tag_image)
            self.image = image
            
            return image
        
    def run(self, name='app'):
        """
        Run a platform with Docker containers.
        
        Parameters:
            name (str): Prefix of a Docker container.
        Returns:
            image (object): Docker container.
        """
        if self.type_app == 'single':
            ports = {'8001/tcp': 8001}  
            host_path = os.path.join(os.path.abspath('.'), 'app_single')
            try:
                print(host_path)
                os.makedirs(host_path)
            except OSError as exception:
                if exception.errno != errno.EEXIST:
                    raise
            container_path = '/root/project'
            volumen = {host_path: {'bind': container_path, 'mode': 'rw'}}
            container = client.containers.run(self.tag_image, name=self.tag_image, 
                                              tty=True, detach=True,
                                              volumes=volumen, ports=ports)
            
            self.containers = [container]
            
            return container

In [140]:
single_file = '/home/guess/Desktop/autodeploy/examples/workflow_sklearn/Dockerfile.app'
# single_file = '/home/guess/Desktop/autodeploy/examples/workflow_sklearn/Dockerfile'
s = Setup(single_file)
s

In [141]:
image = s.build()
image

(<Image: 'app_single:latest'>, <itertools._tee at 0x7f5dd9c01288>)

In [121]:
container = s.run()
container

/home/guess/Desktop/autodeploy/examples/workflow_sklearn/app_single


<Container: 2ce2fcc41b>

In [144]:
container.name

True

In [137]:
container.exec_run(cmd='python test.py')

ExecResult(exit_code=0, output=b'inside python\n')

In [ ]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

# Author: Gusseppe Bravo <gbravor@uni.pe>
# License: BSD 3 clause

import docker
import errno

client = docker.from_env()

class Deploy:
    def __init__(self,
                 container, workflow):
        """
        Example:
            deploy = Deploy(container, workflow)

        Parameters:
            container (object): Container, single mode.
            workflow (dict): Dict of python files.
        
        """    
        self.container = container
        self.workflow = workflow
        
    def run_workflow(self):
        """
        Build a platform with Docker images.
        
        Parameters:
            name (str): Prefix of a Docker image.
        Returns:
            image (object): Docker image.
        """
        if self.type_app == 'single':
            base_path = os.path.dirname(self.single_file)
            self.tag_image = name+'_'+self.type_app
            image = client.images.build(path=base_path, 
                                        dockerfile=self.single_file, 
                                        tag=self.tag_image)
            return image
        
    def deploy(self, name='app'):
        """
        Run a platform with Docker containers.
        
        Parameters:
            name (str): Prefix of a Docker container.
        Returns:
            image (object): Docker container.
        """
        if self.type_app == 'single':
            ports = {'8001/tcp': 8001}  
            host_path = os.path.join(os.path.abspath('.'), 'app_single')
            try:
                print(host_path)
                os.makedirs(host_path)
            except OSError as exception:
                if exception.errno != errno.EEXIST:
                    raise
            container_path = '/root/project'
            volumen = {host_path: {'bind': container_path, 'mode': 'rw'}}
            container = client.containers.run(self.tag_image, name=self.tag_image, 
                                              tty=True, detach=True,
                                              volumes=volumen, ports=ports)
                       
            return container